# 循环神经网络
:label:`sec_rnn`

在:numref:`sec_language_model`中，我们介绍了 $n$ 元语法模型，其中单词 $x_t$ 在时间步 $t$ 的条件概率仅取决于前面 $n-1$ 个单词。对于时间步 $t-(n - 1)$ 之前的单词，如果我们将其可能产生的影响合并到 $x_t$ 上，需要增加 $n$，然而模型参数的数量也会呈指数增长，因为词表 $\mathcal{V}$ 需要存储 $|\mathcal{V}|^n$ 个数字，因此与其将 $$ 模型化，不如使用隐变量模型：
$$
    P(x_t | x_{t-1}, \cdots, x_1) \approx P(x_t | h_{t-1}) 
$$
其中 $h_{t-1}$ 是隐状态(hidden state)，也称为隐藏变量(hidden variable)，它存储了到时间 步 $t-1$ 的序列信息。通常，我们可以基于当前输入 $x_t$ 和先前隐状态 $h_{t-1}$ 来计算时间步 $t$ 处的任何时间的隐状态：
$$
    h_t = f(x_t, h_{t-1})
$$
:eqlabel:`eq_ht_xt`

对于:eqlabel:`eq_ht_xt`中的函数 $f$，隐变量模型不是近似值。毕竟 $h_t$ 是可以仅仅存储到目前为止观察到的所有数据，然而这样的操作可能会使计算和存储的代价都变得很昂贵。

回想一下，我们在:numref:`chap_perceptrons`中讨论过的具有隐藏单元的隐藏层。值得注意的是，隐藏层和隐状态指的是两个截然不同的概念。如上所述，隐藏层是在输入到输出的路径上（以观测角度来理解）的隐藏的层，而隐状态则是在给定步骤所做的任何事情（以技术角度来定义）的输入，并且这些状态只能通过先前时间步的数据来计算。

循环神经网络（RNNs）是具有隐状态的神经网络 。在介绍循环神经网络模型之前，我们首先回顾:numref:`sec_mlp`中介绍的多层感知机模型。

## 无隐状态的神经网络

让我们来看一看只有单隐藏层的多层感知机。设隐藏层的激活函数为 $\phi$，给定一个小批量样本 $\mathbf{X}\in \mathbb{R}^{n \times d}$，其中批量大小为 $n$，输入维度为 $d$，则隐藏层的输出 $\mathbf{H} \in \mathbb{R}^{n \times h}$ 通过下式计算：
$$
    \mathbf{H} = \phi (\mathbf{X} \mathbf{W}_{xh} + \mathbf{b}_h).
$$
:eqlabel:`rnn_h_without_state`

在:eqlabel:`rnn_h_without_state`中，我们拥有的隐藏层权重参数为 $\mathbf{W}_{xh} \in \mathbb{R}^{d \times h}$，偏置参数为 $\mathbf{b}_h \in \mathbb{R}_{1 \times h}$，以及隐藏单元的数目为 $h$ 。因此求和时可以应用广播机制。

接下来，将隐藏变量 $\mathbf{H}$ 用作输入层的输入。输出层由下式给出:
$$
    \mathbf{O} = \mathbf{H} \mathbf{W}_{hq} + \mathbf{b}_{q},
$$
其中，$\mathbf{O} \in \mathbb{R}^{n \times q}$ 是输出变量，$\mathbf{W}_{hq} \in \mathbb{R}^{h \times q}$ 是权重参数，$\mathbf{b}_{q} \in \mathbb{R}^{1 \times q}$ 是输出层的偏置参数。如果是分类问题，我们可以用softmax($\mathbf{O}$)来计算输出类别的概率分布。

这完全类似于之前在:numref:`sec_sequence`中解决的回归问题，因此我们省略了细节。无需多言，只要可以随机选择“特征-标签”对，并且通过自动微分和随机梯度下降能够学习网络参数就可以了。



## 有隐状态的循环神经网络
有了隐状态之后，情况就完全不同了。假设我们在时间步 $t$ 有小批量输入 $\mathbf{X}_t \in \mathbb{R}_{n \times d}$。换言之，对于 $n$ 个序列样本的小批量， $\mathbf{X}_t$ 的每一行对应于来自该序列的时间步 $t$ 处的一个样本。接下来，用 $\mathbf{H}_t \in \mathbb{R}_{n \times d}$ 表示时间步 $t$ 的隐藏变量。与多层感知机不同的是，我们在这里保存了前一个时间步的隐藏变量 $\mathbf{H}_{t-1}$，并引入了一个新的权重参数 $\mathbf{W}_{hh} \in \mathbb{R}^{h \times h}$，来描述如何在当前时间步中使用前一个时间步的隐藏变量。具体的说，当前时间步的隐藏变量由当前时间步的输入以前一个时间步的隐藏变量一起计算得出：
$$
    \mathbf{H}_t = \phi (\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh} + \mathbf{b}_h).
$$
:eqlabel:`rnn_h_with_state`

与:eqref:`rnn_h_without_state`相比，:eqlabel:`rnn_h_with_state`多添加了一项 $\mathbf{H}_{t-1} \mathbf{W}_{hh}$，从而实例化了:eqref:`eq_ht_xt`。从相邻时间步的隐藏变量 $\mathbf{H}_t$ 和 $\mathbf{H}_{t - 1}$ 之间的关系可知，这些变量捕获并保留了序列直到当前时间步的历史信息，就如当前时间步下神经网络的状态或记忆，因此这样的隐藏变量被称为 *隐状态* （hidden state）。由于在当前的时间步中，隐状态使用的定义与前一个时间步中使用的定义相同，因此:eqref:`rnn_h_with_state`的计算是 *循环的*。于是基于循环计算的隐状态神经网络被命名为 *循环神经网络*(recurrent neural network)。在循环神经网络中执行:eqref:`rnn_h_with_state`计算的层，被称为 *循环层*(recurrent layer)。


有许多不同的方法可以构建循环神经网络，